# Install

In [ ]:
!git clone https://github.com/kopyl/PixArt-alpha.git

In [ ]:
%cd PixArt-alpha

In [ ]:
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu117
!pip install -r requirements.txt
!pip install wandb

## Download model

In [ ]:
!python tools/download.py --model_names "PixArt-XL-2-512x512.pth"

## Make dataset out of ImageNet dataset

In [1]:
import os
from tqdm.notebook import tqdm
from datasets import load_dataset
import json
from pathlib import Path

/home/xutd/.conda/envs/dvp2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset_dir = "/NEW_EDS/JJ_Group/xutd/common_datasets/imagenet_256x256/train"
K = 8
for j in range(K):
    json_name = "/NEW_EDS/JJ_Group/xutd/PixArt-alpha/bins/data_info_256train_{}.json".format(str(j))
    pathlist = Path(dataset_dir).glob('**/*.png')
    img_paths = []
    i = 0
    for path in pathlist:
        # because path is object not string
        if i % K == j:
            path_in_str = str(path)
            img_paths.append(path_in_str)
        i += 1
    print(i, len(img_paths))
    data_info = []
    for item in img_paths: 
        width, height = 256, 256
        ratio = 1
        data_info.append({
            "height": height,
            "width": width,
            "ratio": ratio,
            "path": item,
            "prompt": "",
        })
            
    with open(json_name, "w+") as json_file:
        json.dump(data_info, json_file)

1281166 160146
1281166 160146
1281166 160146
1281166 160146
1281166 160146
1281166 160146
1281166 160145
1281166 160145


## Extract features

In [ ]:
!python /workspace/PixArt-alpha/tools/extract_features.py \
    --img_size 256 \
    --json_path "/NEW_EDS/JJ_Group/xutd/PixArt-alpha/bins/data_info_256val.json" \
    --t5_save_root "/NEW_EDS/JJ_Group/xutd/PixArt-alpha/bins/caption_feature_wmask" \
    --vae_save_root "/NEW_EDS/JJ_Group/xutd/PixArt-alpha/bins/img_vae_features" \
    --pretrained_models_dir "/NEW_EDS/JJ_Group/xutd/PixArt-alpha/bins" \
    --dataset_root "/NEW_EDS/JJ_Group/xutd/PixArt-alpha/bins"

In [ ]:
!wandb login REPLACE_THIS_WITH_YOUR_AUTH_TOKEN_OF_WANDB

## Train model

In [ ]:
!python -m torch.distributed.launch \
    train_scripts/train.py \
    /workspace/PixArt-alpha/notebooks/PixArt_xl2_img512_internal_for_pokemon_sample_training.py \
    --work-dir output/trained_model \
    --loss_report_name="train_loss"